In [1]:
import pandas as pd
import numpy as np
import glob
import pickle

import os
from scipy.interpolate import interp1d
import re

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

In [2]:
#ForKK Local
FileSource = "./GeneratedDataDouble/allData.pickle"
NYC_FileSource = "./GeneratedDataDouble/NYC_allData.pickle"
ISR_FileSource = "./GeneratedDataDouble/ISR_allData.pickle"
ImagePath = "./Visualizations/"

In [3]:
OutputColumns = ["ApproachRate", "ApproachRateOther", 
                 "Rel_Pos_Magnitude", 
                 "ScenarioTime", 
                 "Filtered_Accel1","Filtered_Accel2",
                 "Filtered_Steer1","Filtered_Steer2",
                 "1_Head_Center_Distance","2_Head_Center_Distance",
                 'Adjusted_1_Head_Center_Distance', 'Adjusted_2_Head_Center_Distance',
                 "Filtered_1_Head_Velocity_Total","Filtered_2_Head_Velocity_Total",
                 "1_Turn", "2_Turn",
                 "Centerline_Offset_1", "Centerline_Offset_2",
                 '1_Indicator', '2_Indicator',
                 "RelativeRotation"]

In [4]:
FileList = []
with open(FileSource, 'rb') as f:
    FileList = pickle.load(f)

In [5]:
def separate_letters_numbers(s):
    match = re.match(r"([a-zA-Z]+)([0-9]+)", s)
    if match:
        letters, numbers = match.groups()
        return letters.upper(), numbers
    else:
        return s, ""

In [6]:
def extract_scenario_location_run(file_name):
    base_name = os.path.basename(file_name)
    parts = base_name.split('_')
    scenario = parts[0]
    location = separate_letters_numbers(parts[1])[0]
    run = separate_letters_numbers(parts[1])[1] + '_' + parts[2].split('.')[0]  # Remove the file extension
    return scenario, location, run

In [7]:
def adjust_signs_based_on_min_distance(df, distance_col, adjusted_distance_col):
    # Initialize the adjusted distance column
    df[adjusted_distance_col] = df[distance_col]

    # Determine the point of minimum distance
    min_distance_index = df[distance_col].idxmin()
    
    # Adjust distances before the minimum distance (towards the center)
    df.loc[:min_distance_index, adjusted_distance_col] *= -1

    # Ensure only one direction change is applied
    direction_changed = False

    for i in range(1, len(df)):
        # Check the current direction based on the velocity and distance
        if i <= min_distance_index:
            # Should be negative, moving towards the center
            if df.loc[i, adjusted_distance_col] > 0:
                df.loc[i, adjusted_distance_col] *= -1
        else:
            # Should be positive, moving away from the center
            if df.loc[i, adjusted_distance_col] < 0:
                df.loc[i, adjusted_distance_col] *= -1

        # Allow only one change in direction after reaching the minimum distance
        if i > min_distance_index and not direction_changed:
            direction_changed = True

    return df


In [8]:
# Initialize dictionaries to hold data by scenario and location
data_by_scenario = {}
data_by_scenario_location = {}
data_by_run = {} 
data_by_participant = {}

for file_path in FileList:
    if not os.path.exists(file_path):
        print(f"Could not find {file_path}")
        continue
    
    # print(file_path)
    data = pd.read_feather(file_path)
    
    # Adjust distance with sign
    # data['1_Distance_Change'] = data['1_Head_Center_Distance'].diff()
    # data['Adjusted_1_Head_Center_Distance'] = data['1_Head_Center_Distance']
    # data.loc[data['1_Distance_Change'] < 0, 'Adjusted_1_Head_Center_Distance'] *= -1
    
    # data['2_Distance_Change'] = data['2_Head_Center_Distance'].diff()
    # data['Adjusted_2_Head_Center_Distance'] = data['2_Head_Center_Distance']
    # data.loc[data['2_Distance_Change'] < 0, 'Adjusted_2_Head_Center_Distance'] *= -1
    
    # Apply the adjustment to each run
    data = adjust_signs_based_on_min_distance(data, '1_Head_Center_Distance', 'Adjusted_1_Head_Center_Distance')
    data = adjust_signs_based_on_min_distance(data, '2_Head_Center_Distance', 'Adjusted_2_Head_Center_Distance')

    df = data[OutputColumns]
    
    # Extract scenario, location, and run from the file name
    scenario, location, run = extract_scenario_location_run(file_path)
    
    # Store features in dictionaries by run
    data_by_run[scenario + '_' + location + '_' + run] = df
    
    # Store features in dictionaries by scenario
    if scenario not in data_by_scenario:
        data_by_scenario[scenario] = []
    data_by_scenario[scenario].append(df)
    
    # Store features in dictionaries by participant
    participant = location + '_' + run
    if participant not in data_by_participant:
        data_by_participant[participant] = {}
    if scenario not in data_by_participant[participant]:
        data_by_participant[participant][scenario] = []
    data_by_participant[participant][scenario].append(df)
    
    # # Append data to the corresponding scenario and location
    # if scenario not in data_by_scenario_location:
    #     data_by_scenario_location[scenario] = {}
    # if location not in data_by_scenario_location[scenario]:
    #     data_by_scenario_location[scenario][location] = []
    # data_by_scenario_location[scenario][location].append(df)

In [9]:
len(data_by_run), len(data_by_participant), len(data_by_participant['NYC_22_1A']), len(data_by_participant['NYC_24_1A']['CP5'])

(928, 172, 3, 1)

In [10]:
total_rows = sum(df.shape[0] for df in data_by_run.values())
print(f"Total number of rows in all dataframes: {total_rows}")

Total number of rows in all dataframes: 348804


# Feature Engineering


In [76]:
feature_by_participant = pd.DataFrame(data=data_by_participant.keys(), 
                                      columns=['Participant'])
feature_by_participant['Location'] = feature_by_participant['Participant'].apply(lambda x: x.split('_')[0])

In [77]:
variables = [# Acceleration features
            'Max_Accel', # Maximum acceleration
            'Max_Speed', # Maximum speed
            'Free_Road_Delta', # Free-road acceleration parameter
            # Car-following features
            
            # Defeature_valuesceleration features
            'Decel_Point', # Deceleration point
            'Turning_Speed', # Turning speed
            'Decel_Limit', # Deceleration limit
            'Decel_Delta', # Deceleration parameter
            
            # Condition features
            'D2', # CP6 - car B - A
            'D3', # CP2 - car B - A
            'D4', # CP7 - car B - A
            'D5', # CP1 - car A - B
            'D6'  # CP3 - car A - B
            ]

feature_by_participant[variables] = np.nan
feature_values = pd.DataFrame(columns=variables)

## Free-Road Acceleration
- after turning at 30m: AV will accelerate to speed limit


https://www.sciencedirect.com/topics/computer-science/intelligent-driver-model
https://www.researchgate.net/publication/46158245_Enhanced_Intelligent_Driver_Model_to_Access_the_Impact_of_Driving_Strategies_on_Traffic_Capacity 
https://mtreiber.de/MicroApplet/IDM.html

Suppose a vehicle has a preferred speed of $v_{0}$. Suppose the vehicle is moving at a speed v. If there is no traffic on road, the vehicle may show acceleration to attain the best speed proportional to the speed difference with the preferred speed, given by Eq. (21.2)

free-road acceleration strategy $\dot{v}_{free}(v) = a[1−(\frac{v}{v_{0}})^{\delta}]$

The free acceleration is characterized by the desired speed v0, the maximum acceleration a, and the exponent δ characterizing how the acceleration decreases with velocity (δ = 1 corresponds to a linear increase while δ → ∞ denotes a constant acceleration).

### Maximum Speed = 6.362 m/s (median)

In [78]:
for participant, scenarios in data_by_participant.items():
    max_speeds = []
    for scenario, df_list in scenarios.items():
        for df in df_list:
            max_speeds.append(df['Filtered_1_Head_Velocity_Total'].max())
    feature_by_participant.loc[feature_by_participant['Participant'] == participant, ['Max_Speed']] = np.mean(max_speeds)
feature_by_participant

,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,NaN,6.872677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,NaN,7.144224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYC_1_1A,NYC,NaN,5.418505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,NaN,9.589591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,NaN,5.804289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,NaN,4.310480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,ISR_04_1A,ISR,NaN,5.469113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,NaN,2.763167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,NaN,5.034987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
import plotly.express as px
import numpy as np
from scipy.stats import gaussian_kde

fig = px.histogram(feature_by_participant, x='Max_Speed', title='Speed Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
speed_data = feature_by_participant['Max_Speed']
kde = gaussian_kde(speed_data)
x_range = np.linspace(min(speed_data), max(speed_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()


In [80]:
feature_values.loc['Value', 'Max_Speed'] = feature_by_participant['Max_Speed'].mean()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,NaN,6.363749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Maximum Acceleration =1 m/s^2

In [81]:
for participant, scenarios in data_by_participant.items():
    max_accel = []
    for scenario, df_list in scenarios.items():
        for df in df_list:
            max_accel.append(df['Filtered_Accel1'].max())
    print(participant, max_accel)
    feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                               ['Max_Accel']] = np.mean(max_accel) 
    # mean of all runs by one participant
feature_by_participant

NYC_22_1A [0.99994, 0.99958, 0.8216600000000001]
NYC_22_1B [1.0, 0.7188600000000002, 1.0]
NYC_1_1A [0.5793, 0.56052]
NYC_1_1B [1.0, 0.74742]
NYC_25_1A [0.47616, 0.5976, 0.8002600000000001, 0.50366, 0.6326400000000001, 0.3571]
NYC_25_1B [1.0, 0.6998200000000001, 0.6741800000000001, 0.9536199999999999, 0.60396, 0.6365200000000001]
NYC_13_1A [0.91332, 0.80462, 0.8001000000000001, 0.8271200000000001, 0.65168, 0.95194, 0.6780800000000001]
NYC_13_1B [0.8667800000000001, 0.5576400000000001, 0.51642, 0.5399600000000001, 0.5662200000000001, 0.5469, 0.72246]
NYC_8_1A [0.71706, 0.85, 1.0, 0.74906, 0.86092, 0.89918, 0.94154]
NYC_8_1B [0.7620800000000001, 0.781, 0.8899800000000001, 0.7846000000000001, 0.64316, 0.57606, 0.94956]
ISR_25_1A [0.7015600000000001, 0.5964, 0.99064, 0.99996, 0.86514, 0.82774]
ISR_25_1B [0.92936, 0.8247, 0.98112, 0.9492200000000001, 0.61118, 0.9712000000000001]
NYC_40_1A [0.78972, 0.7263000000000001, 0.8612200000000001, 0.6976400000000001, 1.0, 0.8708600000000001]
NYC_40_1B

,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
fig = px.histogram(feature_by_participant, x='Max_Accel', title='Accel Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
accel_data = feature_by_participant['Max_Accel']
kde = gaussian_kde(accel_data)
x_range = np.linspace(min(accel_data), max(accel_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [83]:
feature_values.loc['Value', 'Max_Accel'] = feature_by_participant['Max_Accel'].mean()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,0.741831,6.363749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Free-Road Delta

Suppose a vehicle has a preferred speed of $v_{0}$. Suppose the vehicle is moving at a speed v. If there is no traffic on road, the vehicle may show acceleration to attain the best speed proportional to the speed difference with the preferred speed, given by Eq. (21.2)

free-road acceleration strategy $\dot{v}_{free}(v) = a[1−(\frac{v}{v_{0}})^{\delta}]$

The free acceleration is characterized by the desired speed v0, the maximum acceleration a, and the exponent δ characterizing how the acceleration decreases with velocity (δ = 1 corresponds to a linear increase while δ → ∞ denotes a constant acceleration).

In [84]:
from scipy.optimize import curve_fit

# Define the free-road acceleration function
def free_road_acceleration(a, v_ratio, delta):
    return a * (1 - v_ratio ** delta)

# Fit the free-road acceleration curve for each participant
free_road_dist_threshold = -8

for participant, scenarios in data_by_participant.items():
    # Prepare data for fitting
    acceleration_data = []
    vel_ratio_data = []
    max_accel_data = []
    
    v0_values = feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                               ['Max_Speed']].values.flatten()
    a_values = feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                               ['Max_Accel']].values.flatten()
    
    for scenario, df_list in scenarios.items():
        for df in df_list:
            free_road_idx = df[(df['Adjusted_1_Head_Center_Distance'] < free_road_dist_threshold) # The car is moving towards the center
                               & (df['Filtered_Accel1'] > 0)].index # The car is accelerating
            for i in free_road_idx:
                acceleration_data.extend(df.loc[free_road_idx,'Filtered_Accel1'].values)
                vel_ratio_data.extend(df.loc[free_road_idx, 'Filtered_1_Head_Velocity_Total'].values / v0_values[0])
                max_accel_data.extend([a_values[0]] * len(free_road_idx))

    # Flatten the list of data
    acceleration_data = np.array(acceleration_data).flatten()
    vel_ratio_data = np.array(vel_ratio_data).flatten()
    max_accel_data = np.array(max_accel_data).flatten()
    # print(len(acceleration_data), len(vel_ratio_data), len(max_accel_data))

    if len(acceleration_data) > 0 and len(vel_ratio_data) > 0:
        try:
            # Fit the curve
            popt, pcov = curve_fit(lambda vel_ratio, delta: free_road_acceleration(max_accel_data, vel_ratio, delta), 
                                vel_ratio_data, acceleration_data, bounds=(-np.inf, np.inf))

            # Extract the fitted parameters
            delta_fitted = popt[0]

            # print(f"Fitted parameters: delta = {delta_fitted}")
            feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                                    ['Free_Road_Delta']] = delta_fitted
        except RuntimeError:
            print(f"Could not fit the curve for participant {participant}")
    else:
        print(f"No valid data for participant {participant}")

feature_by_participant

/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_5709/2968309806.py:5: RuntimeWarning:

divide by zero encountered in power



,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
fig = px.histogram(feature_by_participant, x='Free_Road_Delta', title='Free_Road_Delta Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant['Free_Road_Delta'].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [86]:
feature_by_participant['Free_Road_Delta'].describe()

count    172.000000
mean       0.840273
std        1.568772
min        0.044291
25%        0.235098
50%        0.450131
75%        0.808795
max       13.615505
Name: Free_Road_Delta, dtype: float64

In [87]:
feature_values.loc['Value', 'Free_Road_Delta'] = feature_by_participant['Free_Road_Delta'].mean()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,0.741831,6.363749,0.840273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Deceleration Strategy

### 1. Decelerate to desired velocity and prepare for turning: 

deceleration strategy $\dot{v}_{decel}(v) = a[1−(\frac{v}{v_{0}})^{\delta}]$
- desired speed v0
- the deceleration limit a, 
- and the exponent δ characterizing how the deceleration increases with velocity (δ = 1 corresponds to a linear increase while δ → ∞ denotes a constant acceleration).

### Deceleration Point = -15.577 m 

In [88]:
for participant, scenarios in data_by_participant.items():
    decel_points = []
    for scenario, df_list in scenarios.items():
        for df in df_list:
            decel_idx = df[(df['Filtered_Accel1'] < 0) & (df['Filtered_1_Head_Velocity_Total'] > 0)].index
            if decel_idx.empty:
                continue
            decel_point = df.loc[decel_idx[0], 'Adjusted_1_Head_Center_Distance']
            if decel_point < 0:
                decel_points.append(decel_point)
    print(participant, decel_points)
    feature_by_participant.loc[feature_by_participant['Participant'] == participant, ['Decel_Point']] = np.mean(decel_points)
feature_by_participant

NYC_22_1A [-12.467378456596238, -11.245131990777166, -15.347901460786098]
NYC_22_1B [-22.001246659223654, -21.255520899286378, -21.626782641206713]
NYC_1_1A [-13.16270987373041, -13.2986135991689]
NYC_1_1B [-19.160153169011984]
NYC_25_1A [-18.777965867207236, -17.441899660300766]
NYC_25_1B [-9.027269973253265, -12.4960006966229, -12.826388490919804, -13.008974239731586, -16.151650363662533]
NYC_13_1A [-13.625063447191723, -14.800161681549293, -12.688983940804718, -21.92154007409151, -16.493030692992722, -22.809231409453496, -13.809397422045613]
NYC_13_1B [-17.136533649778762, -16.350975225961296, -16.199962638845808, -15.553911739816451, -14.797592326118462, -13.47319873897806, -19.7268448812779]
NYC_8_1A [-13.143906520513601, -19.41102217478513, -14.656930538827016, -11.763364351238977, -14.63812570993978, -13.714282816830051]
NYC_8_1B [-15.114364139453569, -11.90343086425086, -9.3735822634679, -12.791726026615798, -16.096191858946014, -13.422195323418595, -14.41938149575078]
ISR_25_1

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
fig = px.histogram(feature_by_participant, x='Decel_Point', title='Decel_Point Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant['Decel_Point'].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()e

In [90]:
feature_by_participant['Decel_Point'].describe()

count    170.000000
mean     -15.683107
std        3.669189
min      -27.128468
25%      -17.751860
50%      -15.577124
75%      -13.097560
max       -8.013291
Name: Decel_Point, dtype: float64

In [131]:
n = 3
for i in range(n):
    print(i)

0
1
2


In [91]:
feature_values.loc['Value', 'Decel_Point'] = feature_by_participant['Decel_Point'].mean()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,0.741831,6.363749,0.840273,-15.683107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Turning Speed = 1.819034640251255 m/s

In [92]:
for participant, scenarios in data_by_participant.items():
    turning_speeds = []
    for scenario, df_list in scenarios.items():
        for df in df_list:
            turning_idx = df[(df['Adjusted_1_Head_Center_Distance'] < 8) & (df['Adjusted_1_Head_Center_Distance'] > -8)].index
            if turning_idx.empty:
                continue
            turning_speed = df.loc[turning_idx, 'Filtered_1_Head_Velocity_Total'].min()
            if turning_speed > 0.05:
                turning_speeds.append(turning_speed)
    print(participant, turning_speeds)
    feature_by_participant.loc[feature_by_participant['Participant'] == participant, ['Turning_Speed']] = np.mean(turning_speeds)
feature_by_participant

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



NYC_22_1A [2.349674495591393, 1.5657795535573267]
NYC_22_1B [6.413609663205614, 0.20075276191481223]
NYC_1_1A []
NYC_1_1B [6.299552515108125, 8.889079827983831]
NYC_25_1A [4.745960038121183, 3.289892227134121, 0.7922909735177127, 2.0097798985756063, 1.318397831613578, 3.113460462484038]
NYC_25_1B [0.9705698213885083, 2.2405376398225565, 1.0486253460801445, 3.4138158037238617, 2.557107479591943]
NYC_13_1A [2.3405814267470575, 3.2719378228022347, 3.702165628877217, 2.7557535967155773, 3.6177183984792194, 3.561422223829413, 3.115171076950675]
NYC_13_1B [1.1950300412879382, 1.6021971949283278, 1.1426044860528959, 1.7941121089261722, 0.534312858815586]
NYC_8_1A [2.7464413674371504, 2.7182195643132103, 2.5530934817400155, 2.335576572749429, 2.4271924686983395, 2.4069548946385253, 2.841722000167633]
NYC_8_1B [2.9084029169848917, 1.9299635053448767, 0.057984675244420025, 3.6030800963591303, 3.9080410392834075, 4.215012757333651]
ISR_25_1A [1.2366537803107651, 1.7396434181143199, 0.518464071088

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/Users/zhuoqi/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
fig = px.histogram(feature_by_participant, x='Turning_Speed', title='Turning_Speed Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant['Turning_Speed'].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [94]:
feature_values.loc['Value', 'Turning_Speed'] = feature_by_participant['Turning_Speed'].mean()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,0.741831,6.363749,0.840273,-15.683107,1.981169,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Deceleration Limit = -0.588136 m/s^2

In [95]:
for participant, scenarios in data_by_participant.items():
    decel_limit = []
    for scenario, df_list in scenarios.items():
        for df in df_list:
            decel_limit.append(df['Filtered_Accel1'].min())
    feature_by_participant.loc[feature_by_participant['Participant'] == participant, ['Decel_Limit']] = np.mean(decel_limit)
feature_by_participant

,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,NaN,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,NaN,NaN,NaN,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,NaN,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,NaN,NaN,NaN,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,NaN,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
fig = px.histogram(feature_by_participant, x='Decel_Limit', title='Decel_Limit Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant['Decel_Limit'].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [97]:
feature_values.loc['Value', 'Decel_Limit'] = feature_by_participant['Decel_Limit'].mean()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,0.741831,6.363749,0.840273,-15.683107,1.981169,-0.582084,NaN,NaN,NaN,NaN,NaN,NaN


### Deceleration Delta

In [125]:
from scipy.optimize import curve_fit

# Define the free-road acceleration function
def free_road_acceleration(a, v_ratio, delta):
    return a * (1 - v_ratio ** delta)

# Fit the free-road acceleration curve for each participant
free_road_dist_threshold = 0

for participant, scenarios in data_by_participant.items():
    # Prepare data for fitting
    deceleration_data = []
    vel_ratio_data = []
    min_accel_data = []
    
    v0_values = feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                               ['Turning_Speed']].values.flatten()
    
    print(participant, v0_values)
    # Drop rows with empty turning_speed value
    if np.isnan(v0_values[0]):
        feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                                    ['Decel_Delta']] = np.nan
        continue
        
    a_values = feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                               ['Decel_Limit']].values.flatten()
    
    for scenario, df_list in scenarios.items():
        for df in df_list:
            free_road_idx = df[(df['Adjusted_1_Head_Center_Distance'] < free_road_dist_threshold) # The car is moving towards the center
                               & (df['Filtered_Accel1'] < 0)].index # The car is decelerating
            for i in free_road_idx:
                deceleration_data.extend(df.loc[free_road_idx,'Filtered_Accel1'].values)
                vel_ratio_data.extend(df.loc[free_road_idx, 'Filtered_1_Head_Velocity_Total'].values / v0_values[0])
                min_accel_data.extend([a_values[0]] * len(free_road_idx))

    # Flatten the list of data
    deceleration_data = np.array(deceleration_data).flatten()
    vel_ratio_data = np.array(vel_ratio_data).flatten()
    min_accel_data = np.array(min_accel_data).flatten()
    # print(len(acceleration_data), len(vel_ratio_data), len(max_accel_data))

    if len(deceleration_data) > 0 and len(vel_ratio_data) > 0:
        try:
            # Fit the curve
            popt, pcov = curve_fit(lambda vel_ratio, delta: free_road_acceleration(min_accel_data, vel_ratio, delta), 
                                vel_ratio_data, deceleration_data, bounds=(-np.inf, np.inf))

            # Extract the fitted parameters
            delta_fitted = popt[0]

            # print(f"Fitted parameters: delta = {delta_fitted}")
            feature_by_participant.loc[feature_by_participant['Participant'] == participant, 
                                    ['Decel_Delta']] = delta_fitted
        except RuntimeError:
            print(f"Could not fit the curve for participant {participant}")
    else:
        print(f"No valid data for participant {participant}")

feature_by_participant

NYC_22_1A [1.95772702]
NYC_22_1B [3.30718121]
NYC_1_1A [nan]
NYC_1_1B [7.59431617]
NYC_25_1A [2.54496357]
NYC_25_1B [2.04613122]
NYC_13_1A [3.19496431]
NYC_13_1B [1.25365134]
NYC_8_1A [2.57560005]


/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_5709/1098723787.py:5: RuntimeWarning:

divide by zero encountered in power



NYC_8_1B [2.77041417]
ISR_25_1A [1.34478114]


/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_5709/1098723787.py:5: RuntimeWarning:

divide by zero encountered in power

/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_5709/1098723787.py:5: RuntimeWarning:

divide by zero encountered in power



ISR_25_1B [1.25145077]
NYC_40_1A [2.1453024]
NYC_40_1B [2.9214343]
ISR_22_1A [0.98764774]
ISR_22_1B [1.11035082]


...

ISR_02_1A [2.04329154]
ISR_02_1B [2.20770597]
NYC_14_1A [2.05354183]
NYC_14_1B [2.80826707]
ISR_05_1A [2.08101136]
ISR_05_1B [0.47459059]
ISR_33_1A [1.81908714]
ISR_33_1B [2.12901208]
ISR_34_1A [3.49607785]
ISR_34_1B [2.76549865]
NYC_32_1A [1.2209586]


/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_5709/1098723787.py:5: RuntimeWarning:

divide by zero encountered in power



NYC_32_1B [2.8953554]
NYC_35_1A [2.25088464]
NYC_35_1B [3.03800817]
ISR_35_1A [1.94983994]
ISR_35_1B [2.60853205]
ISR_32_1A [2.52077065]
ISR_32_1B [0.79705582]
ISR_04_1A [1.15397723]
ISR_04_1B [nan]
ISR_03_1A [0.31523204]
ISR_03_1B [0.99706115]


/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_5709/1098723787.py:5: RuntimeWarning:

divide by zero encountered in power



,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,0.206765,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,0.257356,NaN,-7.044479,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,52.899173,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,0.627140,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,0.146222,NaN,-10.286774,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,0.104562,NaN,NaN,NaN,NaN,-19.620414
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,0.092908,NaN,NaN,NaN,-12.106047,-10.659093


In [126]:
fig = px.histogram(feature_by_participant, x='Decel_Delta', title='Decel_Delta Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant['Decel_Delta'].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [127]:
feature_by_participant['Decel_Delta'].describe()

count    165.000000
mean       0.612883
std        4.172800
min       -0.012962
25%        0.080392
50%        0.133648
75%        0.231852
max       52.899173
Name: Decel_Delta, dtype: float64

In [100]:
feature_values.loc['Value', 'Decel_Delta'] = feature_by_participant['Decel_Delta'].median()
feature_values

,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
Value,0.741831,6.363749,0.840273,-15.683107,1.981169,-0.582084,0.601388,NaN,NaN,NaN,NaN,NaN


### 2. Decelerate to stop before stop bars: 


## Distance Conditions

### 1. Go straight: No right-turning vehicle from D for the target lane in D2 m; 

 CP6 - car B - A

In [128]:
scenario = 'CP6'
agent = '1B'
condition = 'D2'
# Iterate through each participant and their corresponding scenarios
for participant, scenarios in data_by_participant.items():
    # Check if the scenario 'CP6' is in the scenarios and the participant contains '1B'
    if scenario in scenarios.keys() and agent in participant:
        # Select the specific scenario 'CP6' for analysis
        # Get the list of DataFrames associated with this scenario
        df_list = scenarios[scenario]

        # Iterate through each DataFrame in the list
        for df in df_list:
            # Find indices where the agent car is within the specified distance range
            idx_distance_range = df[(df['Adjusted_1_Head_Center_Distance'] > -15) & 
                                    (df['Adjusted_1_Head_Center_Distance'] < 0)].index
            
            # TODO: Determine when braking events occur by checking for deceleration
            # Identify braking events where deceleration occurs and speed is low
            idx_braking_events = df[(df['Filtered_Accel1'] < 0) & 
                                    (df['Filtered_1_Head_Velocity_Total'] < 1.98)].index

            # Find the intersection of distance range and braking events
            idx_deceleration_point = idx_distance_range.intersection(idx_braking_events)

            # Record the distance between the two cars at the deceleration point
            if not idx_deceleration_point.empty:
                # Record the relative distance between car B and car A
                # D2_AB = df.loc[idx_deceleration_point, 'Rel_Pos_Magnitude'].values[0]
                # Record the distance of car A from the center at the deceleration point
                dist = df.loc[idx_deceleration_point, 'Adjusted_2_Head_Center_Distance'].values[0]
                
                # feature_by_participant.loc[feature_by_participant['Participant'] == participant, 'D2_AB'] = D2_AB
                feature_by_participant.loc[feature_by_participant['Participant'] == participant, condition] = dist

            # Plot the adjusted distance over scenario time for visual inspection
            # plt.plot(df.loc[idx_distance_range, 'ScenarioTime'], df.loc[idx_distance_range, 'Adjusted_1_Head_Center_Distance'], label='Head Center Distance')
            
        # Break after the first participant that meets the conditions (optional)
        # break

# Display the feature_by_participant DataFrame to review stored values
feature_by_participant


,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,0.206765,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,0.257356,NaN,-7.044479,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,NaN,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,52.899173,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,0.627140,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,0.146222,NaN,-10.286774,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,0.104562,NaN,NaN,NaN,NaN,-19.620414
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,NaN,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,0.092908,NaN,NaN,NaN,-12.106047,-10.659093


In [129]:
fig = px.histogram(feature_by_participant, x='D2', title='D2 Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant['D2'].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [103]:
feature_values.loc['Value','D2'] = feature_by_participant['D2'].mean()

In [104]:
# features = ['Filtered_1_Head_Velocity_Total', 
#             'Filtered_2_Head_Velocity_Total',
#             'ApproachRate', 
#             'Adjusted_1_Head_Center_Distance',
#             'Filtered_Accel1',
#             'Filtered_Accel2',
#             'Adjusted_2_Head_Center_Distance']  

# reg_df = pd.DataFrame()
# # Iterate through each participant and their corresponding scenarios
# for participant, scenarios in data_by_participant.items():
#     # Check if the scenario 'CP6' is in the scenarios and the participant contains '1B'
#     if 'CP6' in scenarios.keys() and '1B' in participant:
#         # Select the specific scenario 'CP6' for analysis
#         scenario = 'CP6'
#         # Get the list of DataFrames associated with this scenario
#         df_list = scenarios[scenario]

#         # Iterate through each DataFrame in the list
#         for df in df_list:
#             # Find indices where the agent car is within the specified distance range
#             idx_distance_range = df[(df['Adjusted_1_Head_Center_Distance'] > -15) & 
#                                     (df['Adjusted_1_Head_Center_Distance'] < 0)].index
            
#             reg_df = pd.concat((reg_df, df.loc[idx_distance_range, features]))

# # Display the feature_by_participant DataFrame to review stored values
# reg_df


In [105]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# # Step 1: Prepare the data
# # Define features (X) and the target (y)
# features = [
#             'Filtered_1_Head_Velocity_Total', 
#             'Filtered_2_Head_Velocity_Total',
#             'ApproachRate', 
#             'Adjusted_1_Head_Center_Distance',
#             'Filtered_Accel1',
#             'Filtered_Accel2'
#             ]  
# X = reg_df[features]
# y = reg_df['Adjusted_2_Head_Center_Distance']

# # Step 2: Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Step 3: Choose a regression model
# # Option 1: Linear Regression
# model = LinearRegression()

# # Option 2: Random Forest Regressor
# # model = RandomForestRegressor(n_estimators=100, random_state=42)

# # Step 4: Train the model
# model.fit(X_train, y_train)

# # Step 5: Evaluate the model
# y_pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = mse ** 0.5
# r2 = r2_score(y_test, y_pred)

# print("Model Evaluation Metrics:")
# print(f"Mean Absolute Error (MAE): {mae}")
# print(f"Mean Squared Error (MSE): {mse}")
# print(f"Root Mean Squared Error (RMSE): {rmse}")
# print(f"R-squared (R2): {r2}")

# # Optional: Step 6 - Prediction on new data
# # Assuming new_data is a DataFrame with the same structure as X
# # new_predictions = model.predict(new_data)


<!-- ### 2. Go straight: No left-turning vehicle from D in D3 m; # CP2 - car B - A -->


### 2. Go straight: No left-turning vehicle from D in D3 m; # CP2 - car B - A


In [106]:
scenario = 'CP2'
agent = '1B'
condition = 'D3'
# Iterate through each participant and their corresponding scenarios
for participant, scenarios in data_by_participant.items():
    # Check if the scenario 'CP6' is in the scenarios and the participant contains '1B'
    if scenario in scenarios.keys() and agent in participant:
        # Select the specific scenario 'CP6' for analysis
        # Get the list of DataFrames associated with this scenario
        df_list = scenarios[scenario]

        # Iterate through each DataFrame in the list
        for df in df_list:
            # Find indices where the agent car is within the specified distance range
            idx_distance_range = df[(df['Adjusted_1_Head_Center_Distance'] > -15) & 
                                    (df['Adjusted_1_Head_Center_Distance'] < 0)].index
            
            # TODO: Determine when braking events occur by checking for deceleration
            # Identify braking events where deceleration occurs and speed is low
            idx_braking_events = df[(df['Filtered_Accel1'] < 0) & 
                                    (df['Filtered_1_Head_Velocity_Total'] < 0.05)].index

            # Find the intersection of distance range and braking events
            idx_deceleration_point = idx_distance_range.intersection(idx_braking_events)

            # Record the distance between the two cars at the deceleration point
            if not idx_deceleration_point.empty:
                # Record the relative distance between car B and car A
                # D2_AB = df.loc[idx_deceleration_point, 'Rel_Pos_Magnitude'].values[0]
                # Record the distance of car A from the center at the deceleration point
                dist = df.loc[idx_deceleration_point, 'Adjusted_2_Head_Center_Distance'].values[0]
                
                # feature_by_participant.loc[feature_by_participant['Participant'] == participant, 'D2_AB'] = D2_AB
                feature_by_participant.loc[feature_by_participant['Participant'] == participant, condition] = dist

            # Plot the adjusted distance over scenario time for visual inspection
            # plt.plot(df.loc[idx_distance_range, 'ScenarioTime'], df.loc[idx_distance_range, 'Adjusted_1_Head_Center_Distance'], label='Head Center Distance')
            
        # Break after the first participant that meets the conditions (optional)
        # break

# Display the feature_by_participant DataFrame to review stored values
feature_by_participant


,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,0.206765,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,0.257356,NaN,-7.044479,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,0.143899,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,52.899173,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,0.627140,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,0.146222,NaN,-10.286774,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,0.104562,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,0.320695,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,0.092908,NaN,NaN,NaN,NaN,NaN


In [107]:
fig = px.histogram(feature_by_participant, x=condition, title=condition+' Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant[condition].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [108]:
feature_values.loc['Value',condition] = feature_by_participant[condition].mean()

### 3. Go straight: No go-straight-vehicle from D in D4 m; # CP7 - car B - A


In [109]:
scenario = 'CP7'
agent = '1B'
condition = 'D4'
# Iterate through each participant and their corresponding scenarios
for participant, scenarios in data_by_participant.items():
    # Check if the scenario 'CP6' is in the scenarios and the participant contains '1B'
    if scenario in scenarios.keys() and agent in participant:
        # Select the specific scenario 'CP6' for analysis
        # Get the list of DataFrames associated with this scenario
        df_list = scenarios[scenario]

        # Iterate through each DataFrame in the list
        for df in df_list:
            # Find indices where the agent car is within the specified distance range
            idx_distance_range = df[(df['Adjusted_1_Head_Center_Distance'] > -15) & 
                                    (df['Adjusted_1_Head_Center_Distance'] < 0)].index
            
            # TODO: Determine when braking events occur by checking for deceleration
            # Identify braking events where deceleration occurs and speed is low
            idx_braking_events = df[(df['Filtered_Accel1'] < 0) & 
                                    (df['Filtered_1_Head_Velocity_Total'] < 0.05)].index

            # Find the intersection of distance range and braking events
            idx_deceleration_point = idx_distance_range.intersection(idx_braking_events)

            # Record the distance between the two cars at the deceleration point
            if not idx_deceleration_point.empty:
                # Record the relative distance between car B and car A
                # D2_AB = df.loc[idx_deceleration_point, 'Rel_Pos_Magnitude'].values[0]
                # Record the distance of car A from the center at the deceleration point
                dist = df.loc[idx_deceleration_point, 'Adjusted_2_Head_Center_Distance'].values[0]
                
                # feature_by_participant.loc[feature_by_participant['Participant'] == participant, 'D2_AB'] = D2_AB
                feature_by_participant.loc[feature_by_participant['Participant'] == participant, condition] = dist

            # Plot the adjusted distance over scenario time for visual inspection
            # plt.plot(df.loc[idx_distance_range, 'ScenarioTime'], df.loc[idx_distance_range, 'Adjusted_1_Head_Center_Distance'], label='Head Center Distance')
            
        # Break after the first participant that meets the conditions (optional)
        # break

# Display the feature_by_participant DataFrame to review stored values
feature_by_participant


,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,0.206765,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,0.257356,NaN,-7.044479,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,0.143899,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,52.899173,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,0.627140,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,0.146222,NaN,-10.286774,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,0.104562,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,0.320695,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,0.092908,NaN,NaN,NaN,NaN,NaN


In [110]:
fig = px.histogram(feature_by_participant, x=condition, title=condition+' Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant[condition].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [111]:
feature_values.loc['Value',condition] = feature_by_participant[condition].mean()

### 4. Left turn: No go-straight vehicle from C for the target lane in D5 m; # CP1 - car A - B


In [112]:
scenario = 'CP1'
agent = '1A'
condition = 'D5'
# Iterate through each participant and their corresponding scenarios
for participant, scenarios in data_by_participant.items():
    # Check if the scenario 'CP6' is in the scenarios and the participant contains '1B'
    if scenario in scenarios.keys() and agent in participant:
        # Select the specific scenario 'CP6' for analysis
        # Get the list of DataFrames associated with this scenario
        df_list = scenarios[scenario]

        # Iterate through each DataFrame in the list
        for df in df_list:
            # Find indices where the agent car is within the specified distance range
            idx_distance_range = df[(df['Adjusted_1_Head_Center_Distance'] > -15) & 
                                    (df['Adjusted_1_Head_Center_Distance'] < 0)].index
            
            # TODO: Determine when braking events occur by checking for deceleration
            # Identify braking events where deceleration occurs and speed is low
            idx_braking_events = df[(df['Filtered_Accel1'] < 0) & 
                                    (df['Filtered_1_Head_Velocity_Total'] < 0.05)].index

            # Find the intersection of distance range and braking events
            idx_deceleration_point = idx_distance_range.intersection(idx_braking_events)

            # Record the distance between the two cars at the deceleration point
            if not idx_deceleration_point.empty:
                # Record the relative distance between car B and car A
                # D2_AB = df.loc[idx_deceleration_point, 'Rel_Pos_Magnitude'].values[0]
                # Record the distance of car A from the center at the deceleration point
                dist = df.loc[idx_deceleration_point, 'Adjusted_2_Head_Center_Distance'].values[0]
                
                # feature_by_participant.loc[feature_by_participant['Participant'] == participant, 'D2_AB'] = D2_AB
                feature_by_participant.loc[feature_by_participant['Participant'] == participant, condition] = dist

            # Plot the adjusted distance over scenario time for visual inspection
            # plt.plot(df.loc[idx_distance_range, 'ScenarioTime'], df.loc[idx_distance_range, 'Adjusted_1_Head_Center_Distance'], label='Head Center Distance')
            
        # Break after the first participant that meets the conditions (optional)
        # break

# Display the feature_by_participant DataFrame to review stored values
feature_by_participant


,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,0.206765,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,0.257356,NaN,-7.044479,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,0.143899,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,52.899173,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,0.627140,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,0.146222,NaN,-10.286774,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,0.104562,NaN,NaN,NaN,NaN,NaN
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,0.320695,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,0.092908,NaN,NaN,NaN,-12.106047,NaN


In [113]:
fig = px.histogram(feature_by_participant, x=condition, title=condition+' Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant[condition].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [114]:
feature_values.loc['Value',condition] = feature_by_participant[condition].mean()

### 5. Left turn: No left-turning vehicle from D in D6 m; # CP3 - car A - B

In [115]:
scenario = 'CP3'
agent = '1A'
condition = 'D6'
# Iterate through each participant and their corresponding scenarios
for participant, scenarios in data_by_participant.items():
    # Check if the scenario 'CP6' is in the scenarios and the participant contains '1B'
    if scenario in scenarios.keys() and agent in participant:
        # Select the specific scenario 'CP6' for analysis
        # Get the list of DataFrames associated with this scenario
        df_list = scenarios[scenario]

        # Iterate through each DataFrame in the list
        for df in df_list:
            # Find indices where the agent car is within the specified distance range
            idx_distance_range = df[(df['Adjusted_1_Head_Center_Distance'] > -15) & 
                                    (df['Adjusted_1_Head_Center_Distance'] < 0)].index
            
            # TODO: Determine when braking events occur by checking for deceleration
            # Identify braking events where deceleration occurs and speed is low
            idx_braking_events = df[(df['Filtered_Accel1'] < 0) & 
                                    (df['Filtered_1_Head_Velocity_Total'] < 0.05)].index

            # Find the intersection of distance range and braking events
            idx_deceleration_point = idx_distance_range.intersection(idx_braking_events)

            # Record the distance between the two cars at the deceleration point
            if not idx_deceleration_point.empty:
                # Record the relative distance between car B and car A
                # D2_AB = df.loc[idx_deceleration_point, 'Rel_Pos_Magnitude'].values[0]
                # Record the distance of car A from the center at the deceleration point
                dist = df.loc[idx_deceleration_point, 'Adjusted_2_Head_Center_Distance'].values[0]
                
                # feature_by_participant.loc[feature_by_participant['Participant'] == participant, 'D2_AB'] = D2_AB
                feature_by_participant.loc[feature_by_participant['Participant'] == participant, condition] = dist

            # Plot the adjusted distance over scenario time for visual inspection
            # plt.plot(df.loc[idx_distance_range, 'ScenarioTime'], df.loc[idx_distance_range, 'Adjusted_1_Head_Center_Distance'], label='Head Center Distance')
            
        # Break after the first participant that meets the conditions (optional)
        # break

# Display the feature_by_participant DataFrame to review stored values
feature_by_participant


,Participant,Location,Max_Accel,Max_Speed,Free_Road_Delta,Decel_Point,Turning_Speed,Decel_Limit,Decel_Delta,D2,D3,D4,D5,D6
0,NYC_22_1A,NYC,0.940393,6.872677,0.515552,-13.020137,1.957727,-0.485927,0.206765,NaN,NaN,NaN,NaN,NaN
1,NYC_22_1B,NYC,0.906287,7.144224,0.718190,-21.627850,3.307181,-0.571247,0.257356,NaN,-7.044479,NaN,NaN,NaN
2,NYC_1_1A,NYC,0.569910,5.418505,5.539372,-13.230662,NaN,-0.940080,0.143899,NaN,NaN,NaN,NaN,NaN
3,NYC_1_1B,NYC,0.873710,9.589591,13.615505,-19.160153,7.594316,-0.742910,52.899173,NaN,NaN,NaN,NaN,NaN
4,NYC_25_1A,NYC,0.561237,5.804289,1.188655,-18.109933,2.544964,-0.343933,0.627140,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ISR_32_1B,ISR,0.572207,4.310480,0.467772,-15.024826,0.797056,-0.411733,0.146222,NaN,-10.286774,NaN,NaN,NaN
168,ISR_04_1A,ISR,0.579885,5.469113,0.265135,-10.781218,1.153977,-0.548180,0.104562,NaN,NaN,NaN,NaN,-19.620414
169,ISR_04_1B,ISR,0.385675,2.763167,0.324014,-22.667824,NaN,-0.228350,0.320695,NaN,NaN,NaN,NaN,NaN
170,ISR_03_1A,ISR,0.612784,5.034987,0.079713,-19.890562,0.315232,-0.567344,0.092908,NaN,NaN,NaN,-12.106047,-10.659093


In [116]:
fig = px.histogram(feature_by_participant, x=condition, title=condition+' Distribution', 
                   marginal='violin', histnorm='density')

# Calculate KDE
delta_data = feature_by_participant[condition].dropna()
kde = gaussian_kde(delta_data)
x_range = np.linspace(min(delta_data), max(delta_data), 1000)
fig.add_trace(go.Scatter(x=x_range, y=kde.evaluate(x_range)*100, mode='lines', name='KDE'))

fig.show()

In [117]:
feature_values.loc['Value',condition] = feature_by_participant[condition].mean()

In [118]:
feature_values = feature_values.T

In [119]:
feature_values.to_csv('feature_values.csv')